In [1]:
from new_attempt.stateless import *

In [2]:
from custom_libraries.utilities import *

In [3]:
import matplotlib.pyplot as plt

In [4]:
import numpy as np

In [5]:
regionSymbols = RegionSymbols(1)

In [6]:
customPlanck = sq.Symbol('hbarX', real = True, positive = True, nonzero = True)

In [7]:
waveEquation = constantPotentialTimeIndependentSchroedingerEquation1D(regionSymbols, reducedPlanckConstant = customPlanck)

In [8]:
waveEquation

Eq(V_1*psi_1(x) - hbarX**2*Derivative(psi_1(x), (x, 2))/(2*m), E_{total}*psi_1(x))

In [9]:
normalization = simpleWaveFunctionNormalization(regionSymbols.startDistance, regionSymbols.distance, regionSymbols)

In [10]:
normalization

Eq(Integral(psi_1(x)*conjugate(psi_1(x)), (x, L_0, L_1)), N_1)

In [11]:
boundries = {
        regionSymbols.waveFunction(regionSymbols.startDistance) : sp.Symbol('B_{1}'), 
        regionSymbols.waveFunction(regionSymbols.distance) : sp.Symbol('B_{2}')
    }

In [12]:
generalSolution = sp.dsolve(waveEquation, ics = boundries)

In [13]:
generalSolution

Eq(psi_1(x), (B_{1}*exp(sqrt(2)*L_0*sqrt(m)*sqrt(-E_{total} + V_1)/hbarX)/(exp(2*sqrt(2)*L_0*sqrt(m)*sqrt(-E_{total} + V_1)/hbarX) - exp(2*sqrt(2)*L_1*sqrt(m)*sqrt(-E_{total} + V_1)/hbarX)) - B_{2}*exp(sqrt(2)*L_1*sqrt(m)*sqrt(-E_{total} + V_1)/hbarX)/(exp(2*sqrt(2)*L_0*sqrt(m)*sqrt(-E_{total} + V_1)/hbarX) - exp(2*sqrt(2)*L_1*sqrt(m)*sqrt(-E_{total} + V_1)/hbarX)))*exp(sqrt(2)*sqrt(m)*x*sqrt(-E_{total} + V_1)/hbarX) + (-B_{1}*exp(sqrt(2)*L_0*sqrt(m)*sqrt(-E_{total} + V_1)/hbarX)*exp(2*sqrt(2)*L_1*sqrt(m)*sqrt(-E_{total} + V_1)/hbarX)/(exp(2*sqrt(2)*L_0*sqrt(m)*sqrt(-E_{total} + V_1)/hbarX) - exp(2*sqrt(2)*L_1*sqrt(m)*sqrt(-E_{total} + V_1)/hbarX)) + B_{2}*exp(2*sqrt(2)*L_0*sqrt(m)*sqrt(-E_{total} + V_1)/hbarX)*exp(sqrt(2)*L_1*sqrt(m)*sqrt(-E_{total} + V_1)/hbarX)/(exp(2*sqrt(2)*L_0*sqrt(m)*sqrt(-E_{total} + V_1)/hbarX) - exp(2*sqrt(2)*L_1*sqrt(m)*sqrt(-E_{total} + V_1)/hbarX)))*exp(-sqrt(2)*sqrt(m)*x*sqrt(-E_{total} + V_1)/hbarX))

In [14]:
harmonicConstant = extractHarmonicConstant(regionSymbols, waveEquation)

In [15]:
harmonicConstant

Eq(k_1, sqrt(2)*sqrt(m)*sqrt(-E_{total} + V_1)/hbarX)

In [16]:
generalSolution = generalSolution.subs({harmonicConstant.rhs : harmonicConstant.lhs})

In [17]:
generalSolution

Eq(psi_1(x), (B_{1}*exp(L_0*k_1)/(exp(2*L_0*k_1) - exp(2*L_1*k_1)) - B_{2}*exp(L_1*k_1)/(exp(2*L_0*k_1) - exp(2*L_1*k_1)))*exp(k_1*x) + (-B_{1}*exp(L_0*k_1)*exp(2*L_1*k_1)/(exp(2*L_0*k_1) - exp(2*L_1*k_1)) + B_{2}*exp(2*L_0*k_1)*exp(L_1*k_1)/(exp(2*L_0*k_1) - exp(2*L_1*k_1)))*exp(-k_1*x))

In [18]:
def extractWaveCoefficents(exponentialEquation, position = POSITION_SYMBOL): 
    C0 = sp.Wild("C0")
    C1 = sp.Wild("C1")
    harmonic = sp.Wild("k")
    results = exponentialEquation.match(C0 * sp.exp(harmonic * position) + C1 * sp.exp(-harmonic * position))
    return {
            "transmission" : results[C0], 
            "reflection" : results[C1], 
            "harmonicConstant" : results[harmonic]
    }

In [19]:
coefficeints = extractWaveCoefficents(generalSolution.rhs, POSITION_SYMBOL)

In [20]:
coefficeints

{'transmission': B_{1}*exp(L_0*k_1)/(exp(2*L_0*k_1) - exp(2*L_1*k_1)) - B_{2}*exp(L_1*k_1)/(exp(2*L_0*k_1) - exp(2*L_1*k_1)),
 'reflection': -B_{1}*exp(L_0*k_1)*exp(2*L_1*k_1)/(exp(2*L_0*k_1) - exp(2*L_1*k_1)) + B_{2}*exp(2*L_0*k_1)*exp(L_1*k_1)/(exp(2*L_0*k_1) - exp(2*L_1*k_1)),
 'harmonicConstant': k_1}

In [21]:
exponential = generalSolution.subs({
        coefficeints["transmission"] : sp.Symbol("C0"), 
        coefficeints["reflection"] : sp.Symbol("C1")
    })

In [22]:
exponential

Eq(psi_1(x), C0*exp(k_1*x) + C1*exp(-k_1*x))

In [23]:
coefficeints["transmission"]

B_{1}*exp(L_0*k_1)/(exp(2*L_0*k_1) - exp(2*L_1*k_1)) - B_{2}*exp(L_1*k_1)/(exp(2*L_0*k_1) - exp(2*L_1*k_1))

In [24]:
coefficeints["reflection"]

-B_{1}*exp(L_0*k_1)*exp(2*L_1*k_1)/(exp(2*L_0*k_1) - exp(2*L_1*k_1)) + B_{2}*exp(2*L_0*k_1)*exp(L_1*k_1)/(exp(2*L_0*k_1) - exp(2*L_1*k_1))

In [25]:
integration = normalization.subs({regionSymbols.waveFunction(POSITION_SYMBOL) : exponential.rhs}).doit()

In [26]:
integration

Eq(-L_0*(C0*conjugate(C1) + C1*conjugate(C0)) + L_1*(C0*conjugate(C1) + C1*conjugate(C0)) + Piecewise((-(2*C0*k_1*exp(2*L_0*k_1)*conjugate(C0) - 2*C1*k_1*exp(-2*L_0*k_1)*conjugate(C1))/(4*k_1**2) + (2*C0*k_1*exp(2*L_1*k_1)*conjugate(C0) - 2*C1*k_1*exp(-2*L_1*k_1)*conjugate(C1))/(4*k_1**2), Ne(k_1, 0)), (-L_0*(C0*conjugate(C0) + C1*conjugate(C1)) + L_1*(C0*conjugate(C0) + C1*conjugate(C1)), True)), N_1)

In [27]:
integration.lhs

-L_0*(C0*conjugate(C1) + C1*conjugate(C0)) + L_1*(C0*conjugate(C1) + C1*conjugate(C0)) + Piecewise((-(2*C0*k_1*exp(2*L_0*k_1)*conjugate(C0) - 2*C1*k_1*exp(-2*L_0*k_1)*conjugate(C1))/(4*k_1**2) + (2*C0*k_1*exp(2*L_1*k_1)*conjugate(C0) - 2*C1*k_1*exp(-2*L_1*k_1)*conjugate(C1))/(4*k_1**2), Ne(k_1, 0)), (-L_0*(C0*conjugate(C0) + C1*conjugate(C1)) + L_1*(C0*conjugate(C0) + C1*conjugate(C1)), True))

In [28]:
integration.lhs

-L_0*(C0*conjugate(C1) + C1*conjugate(C0)) + L_1*(C0*conjugate(C1) + C1*conjugate(C0)) + Piecewise((-(2*C0*k_1*exp(2*L_0*k_1)*conjugate(C0) - 2*C1*k_1*exp(-2*L_0*k_1)*conjugate(C1))/(4*k_1**2) + (2*C0*k_1*exp(2*L_1*k_1)*conjugate(C0) - 2*C1*k_1*exp(-2*L_1*k_1)*conjugate(C1))/(4*k_1**2), Ne(k_1, 0)), (-L_0*(C0*conjugate(C0) + C1*conjugate(C1)) + L_1*(C0*conjugate(C0) + C1*conjugate(C1)), True))

In [29]:
def extractNonZero(symbolic):
    harmonic = sp.Wild('k')
    expression = sp.Wild('C')
    nonZero = sp.Ne(harmonic, 0)
    otherExpression = sp.Wild('A')
    otherCondition = sp.Wild('B')
    otherStuff = sp.Wild('Q')
    extracted = symbolic.match(otherStuff + sp.Piecewise((expression, nonZero), (otherExpression, otherCondition)))
    return {
            "nonZero" : extracted[expression], 
            "otherwise" : extracted[otherExpression], 
            "rest" : extracted[otherStuff]
        }

In [30]:
extractedConditions = extractNonZero(integration.lhs)

In [31]:
nonZeroNormalizationCase = sp.Eq(extractedConditions["rest"] + extractedConditions["nonZero"], integration.rhs)

In [32]:
nonZeroNormalizationCase

Eq(-L_0*(C0*conjugate(C1) + C1*conjugate(C0)) + L_1*(C0*conjugate(C1) + C1*conjugate(C0)) - (2*C0*k_1*exp(2*L_0*k_1)*conjugate(C0) - 2*C1*k_1*exp(-2*L_0*k_1)*conjugate(C1))/(4*k_1**2) + (2*C0*k_1*exp(2*L_1*k_1)*conjugate(C0) - 2*C1*k_1*exp(-2*L_1*k_1)*conjugate(C1))/(4*k_1**2), N_1)

In [33]:
c0solutions = sp.solve(nonZeroNormalizationCase, sp.Symbol("C0"))

In [34]:
len(c0solutions)

2

In [35]:
c0solutions[0]

(-2*C1*L_0*k_1*exp(k_1*(L_0 + L_1)) + 2*C1*L_1*k_1*exp(k_1*(L_0 + L_1)) - sqrt(4*C1**2*L_0**2*k_1**2*exp(2*k_1*(L_0 + L_1)) - 8*C1**2*L_0*L_1*k_1**2*exp(2*k_1*(L_0 + L_1)) + 4*C1**2*L_1**2*k_1**2*exp(2*k_1*(L_0 + L_1)) - C1**2*exp(4*L_0*k_1) - C1**2*exp(4*L_1*k_1) + 2*C1**2*exp(2*k_1*(L_0 + L_1)) + 2*N_1*k_1*exp(2*k_1*(L_0 + 2*L_1)) - 2*N_1*k_1*exp(2*k_1*(2*L_0 + L_1))))*exp(-k_1*(L_0 + L_1))/(exp(2*L_0*k_1) - exp(2*L_1*k_1))

In [36]:
c0solutions[1]

(-2*C1*L_0*k_1*exp(k_1*(L_0 + L_1)) + 2*C1*L_1*k_1*exp(k_1*(L_0 + L_1)) + sqrt(4*C1**2*L_0**2*k_1**2*exp(2*k_1*(L_0 + L_1)) - 8*C1**2*L_0*L_1*k_1**2*exp(2*k_1*(L_0 + L_1)) + 4*C1**2*L_1**2*k_1**2*exp(2*k_1*(L_0 + L_1)) - C1**2*exp(4*L_0*k_1) - C1**2*exp(4*L_1*k_1) + 2*C1**2*exp(2*k_1*(L_0 + L_1)) + 2*N_1*k_1*exp(2*k_1*(L_0 + 2*L_1)) - 2*N_1*k_1*exp(2*k_1*(2*L_0 + L_1))))*exp(-k_1*(L_0 + L_1))/(exp(2*L_0*k_1) - exp(2*L_1*k_1))

In [37]:
c1solutions = sp.solve(nonZeroNormalizationCase, sp.Symbol("C1"))

In [38]:
len(c1solutions)

2

In [39]:
c1solutions[0]

(-2*C0*L_0*k_1*exp(k_1*(L_0 + L_1)) + 2*C0*L_1*k_1*exp(k_1*(L_0 + L_1)) - sqrt(4*C0**2*L_0**2*k_1**2*exp(2*k_1*(L_0 + L_1)) - 8*C0**2*L_0*L_1*k_1**2*exp(2*k_1*(L_0 + L_1)) + 4*C0**2*L_1**2*k_1**2*exp(2*k_1*(L_0 + L_1)) - C0**2*exp(4*L_0*k_1) - C0**2*exp(4*L_1*k_1) + 2*C0**2*exp(2*k_1*(L_0 + L_1)) - 2*N_1*k_1*exp(2*L_0*k_1) + 2*N_1*k_1*exp(2*L_1*k_1)))*exp(k_1*(L_0 + L_1))/(exp(2*L_0*k_1) - exp(2*L_1*k_1))

In [40]:
c1solutions[1]

(-2*C0*L_0*k_1*exp(k_1*(L_0 + L_1)) + 2*C0*L_1*k_1*exp(k_1*(L_0 + L_1)) + sqrt(4*C0**2*L_0**2*k_1**2*exp(2*k_1*(L_0 + L_1)) - 8*C0**2*L_0*L_1*k_1**2*exp(2*k_1*(L_0 + L_1)) + 4*C0**2*L_1**2*k_1**2*exp(2*k_1*(L_0 + L_1)) - C0**2*exp(4*L_0*k_1) - C0**2*exp(4*L_1*k_1) + 2*C0**2*exp(2*k_1*(L_0 + L_1)) - 2*N_1*k_1*exp(2*L_0*k_1) + 2*N_1*k_1*exp(2*L_1*k_1)))*exp(k_1*(L_0 + L_1))/(exp(2*L_0*k_1) - exp(2*L_1*k_1))

In [41]:
c0Equation = sp.Eq(coefficeints["transmission"] * coefficeints["transmission"], (c0solutions[0] * c0solutions[1]).simplify())

In [42]:
c0Equation

Eq((B_{1}*exp(L_0*k_1)/(exp(2*L_0*k_1) - exp(2*L_1*k_1)) - B_{2}*exp(L_1*k_1)/(exp(2*L_0*k_1) - exp(2*L_1*k_1)))**2, (C1**2*exp(2*L_0*k_1) - C1**2*exp(2*L_1*k_1) + 2*N_1*k_1*exp(2*k_1*(L_0 + L_1)))*exp(-2*k_1*(L_0 + L_1))/(exp(2*L_0*k_1) - exp(2*L_1*k_1)))

In [43]:
c1solutions.append(sp.solve(c0Equation, sp.Symbol("C1")))

In [44]:
len(c1solutions[2])

2

In [45]:
c1solutions[2][0]

-sqrt(B_{1}**2*exp(2*L_0*k_1) - 2*B_{1}*B_{2}*exp(k_1*(L_0 + L_1)) + B_{2}**2*exp(2*L_1*k_1) - 2*N_1*k_1*exp(2*L_0*k_1) + 2*N_1*k_1*exp(2*L_1*k_1))*exp(k_1*(L_0 + L_1))/(exp(2*L_0*k_1) - exp(2*L_1*k_1))

In [46]:
c1solutions[2][1]

sqrt(B_{1}**2*exp(2*L_0*k_1) - 2*B_{1}*B_{2}*exp(k_1*(L_0 + L_1)) + B_{2}**2*exp(2*L_1*k_1) - 2*N_1*k_1*exp(2*L_0*k_1) + 2*N_1*k_1*exp(2*L_1*k_1))*exp(k_1*(L_0 + L_1))/(exp(2*L_0*k_1) - exp(2*L_1*k_1))

In [47]:
c1ConstrainedSolution = sp.Eq(sp.Symbol("C1") ** 2, (c1solutions[2][0] * c1solutions[2][1]).simplify().refine())

In [48]:
c1ConstrainedSolution

Eq(C1**2, (-B_{1}**2*exp(2*L_0*k_1) + 2*B_{1}*B_{2}*exp(k_1*(L_0 + L_1)) - B_{2}**2*exp(2*L_1*k_1) + 2*N_1*k_1*exp(2*L_0*k_1) - 2*N_1*k_1*exp(2*L_1*k_1))*exp(2*k_1*(L_0 + L_1))/(exp(2*L_0*k_1) - exp(2*L_1*k_1))**2)

In [49]:
c1Equation = sp.Eq(coefficeints["reflection"] * coefficeints["reflection"], (c1solutions[0] * c1solutions[1]).simplify())

In [50]:
c1Equation

Eq((-B_{1}*exp(L_0*k_1)*exp(2*L_1*k_1)/(exp(2*L_0*k_1) - exp(2*L_1*k_1)) + B_{2}*exp(2*L_0*k_1)*exp(L_1*k_1)/(exp(2*L_0*k_1) - exp(2*L_1*k_1)))**2, (C0**2*exp(2*L_0*k_1) - C0**2*exp(2*L_1*k_1) + 2*N_1*k_1)*exp(2*k_1*(L_0 + L_1))/(exp(2*L_0*k_1) - exp(2*L_1*k_1)))

In [51]:
c0solutions.append(sp.solve(c1Equation, sp.Symbol("C0")))

In [52]:
len(c0solutions[2])

2

In [53]:
c0solutions[2][0]

-sqrt(B_{1}**2*exp(2*L_1*k_1) - 2*B_{1}*B_{2}*exp(k_1*(L_0 + L_1)) + B_{2}**2*exp(2*L_0*k_1) - 2*N_1*k_1*exp(2*L_0*k_1) + 2*N_1*k_1*exp(2*L_1*k_1))/(exp(2*L_0*k_1) - exp(2*L_1*k_1))

In [54]:
c0solutions[2][1]

sqrt(B_{1}**2*exp(2*L_1*k_1) - 2*B_{1}*B_{2}*exp(k_1*(L_0 + L_1)) + B_{2}**2*exp(2*L_0*k_1) - 2*N_1*k_1*exp(2*L_0*k_1) + 2*N_1*k_1*exp(2*L_1*k_1))/(exp(2*L_0*k_1) - exp(2*L_1*k_1))

In [88]:
c0ConstrainedSolution = sp.Eq(sp.Symbol("C0") ** 2, (c0solutions[2][0] * c0solutions[2][1]).simplify().refine())

In [89]:
c0ConstrainedSolution

Eq(C0**2, (-B_{1}**2*exp(2*L_1*k_1) + 2*B_{1}*B_{2}*exp(k_1*(L_0 + L_1)) - B_{2}**2*exp(2*L_0*k_1) + 2*N_1*k_1*exp(2*L_0*k_1) - 2*N_1*k_1*exp(2*L_1*k_1))/(exp(2*L_0*k_1) - exp(2*L_1*k_1))**2)

In [90]:
(exponential.rhs * sp.conjugate(exponential.rhs)).expand()

C0*exp(2*k_1*x)*conjugate(C0) + C0*conjugate(C1) + C1*conjugate(C0) + C1*exp(-2*k_1*x)*conjugate(C1)

In [92]:
c0solutions[2][0]

-sqrt(B_{1}**2*exp(2*L_1*k_1) - 2*B_{1}*B_{2}*exp(k_1*(L_0 + L_1)) + B_{2}**2*exp(2*L_0*k_1) - 2*N_1*k_1*exp(2*L_0*k_1) + 2*N_1*k_1*exp(2*L_1*k_1))/(exp(2*L_0*k_1) - exp(2*L_1*k_1))

In [93]:
c0solutions[2][1]

sqrt(B_{1}**2*exp(2*L_1*k_1) - 2*B_{1}*B_{2}*exp(k_1*(L_0 + L_1)) + B_{2}**2*exp(2*L_0*k_1) - 2*N_1*k_1*exp(2*L_0*k_1) + 2*N_1*k_1*exp(2*L_1*k_1))/(exp(2*L_0*k_1) - exp(2*L_1*k_1))

In [96]:
c0solutions[2][1] + c0solutions[2][0]

0

In [157]:
def extractCoefficientsFromComplexPartOfQuadraticEquation(complexPart): 
    A = sp.Wild("A")
    B = sp.Wild("Q")
    C = sp.Wild("C")
    pattern = sp.sqrt(4 * A * C) / (2 * A)
    equation = sp.Eq(complexPart, pattern)
    return {
        "C" : sp.solve(equation, C), 
        "B" : sp.solve(equation, B), 
        "A" : sp.solve(equation, A), 
        "match" : complexPart.match(pattern)
    }

In [158]:
solutions = extractCoefficientsFromComplexPartOfQuadraticEquation(c0solutions[2][0])

In [159]:
solutions["A"][0]

C_*(exp(4*L_0*k_1) + exp(4*L_1*k_1) - 2*exp(2*k_1*(L_0 + L_1)))/(B_{1}**2*exp(2*L_1*k_1) - 2*B_{1}*B_{2}*exp(k_1*(L_0 + L_1)) + B_{2}**2*exp(2*L_0*k_1) - 2*N_1*k_1*exp(2*L_0*k_1) + 2*N_1*k_1*exp(2*L_1*k_1))

In [165]:
solutions["B"]

[]

In [162]:
solutions["C"][0]

A_*(B_{1}**2*exp(2*L_1*k_1) - 2*B_{1}*B_{2}*exp(k_1*(L_0 + L_1)) + B_{2}**2*exp(2*L_0*k_1) - 2*N_1*k_1*exp(2*L_0*k_1) + 2*N_1*k_1*exp(2*L_1*k_1))/((exp(L_0*k_1) - exp(L_1*k_1))**2*(exp(L_0*k_1) + exp(L_1*k_1))**2)

In [150]:
solutions["match"]

In [154]:
AWild = sp.Wild("A")

In [156]:
sp.solve(sp.Eq(AWild, solutions["A"][0].subs({sp.Wild("C") : solutions["C"][0]})), AWild)

[0]